In [1]:
import geopandas as gpd
import numpy as np
import folium
from scipy.spatial.distance import cdist


In [23]:

# 读取GeoJSON文件
geojson_path = './combined_coordinates.geojson'
gdf = gpd.read_file(geojson_path)


In [ ]:

# 提取坐标点
coordinates = gdf.geometry.apply(lambda geom: (geom.x, geom.y)).tolist()

# 计算距离矩阵
def haversine(coord1, coord2):
    lat1, lon1 = np.radians(coord1)
    lat2, lon2 = np.radians(coord2)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    r = 6371  # Radius of earth in kilometers
    return c * r

distance_matrix = cdist(coordinates, coordinates, metric=lambda u, v: haversine(u, v))

# 创建地图
m = folium.Map(location=[np.mean([coord[1] for coord in coordinates]), np.mean([coord[0] for coord in coordinates])], zoom_start=12)

# 添加点和距离线
for i, coord1 in enumerate(coordinates):
    for j, coord2 in enumerate(coordinates):
        if i < j:  # 只计算一次每对距离
            distance = distance_matrix[i, j]
            folium.PolyLine([coord1, coord2], color="blue", weight=1.5, opacity=0.7, tooltip=f'{distance:.2f} km').add_to(m)
    folium.Marker(location=list(coord1), popup=f'Point {i+1}').add_to(m)

# 保存地图为HTML文件
map_path = './distance_map.html'
m.save(map_path)
map_path


In [7]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin

# Define the URL of the HTML file
html_file = './distance_map.html'

# Define the directory to save the downloaded files
save_directory = './resources'

# Create the save directory if it doesn't exist
os.makedirs(save_directory, exist_ok=True)

# Read the HTML file
with open(html_file, 'r') as f:
    html_content = f.read()

# Parse the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Find all the <a> tags with href attribute
links = soup.find_all('a', href=True)

# Iterate over the links
for link in links:
    # Get the URL of the linked file
    file_url = link['href']
    
    # Parse the URL to get the filename
    parsed_url = urlparse(file_url)
    filename = os.path.basename(parsed_url.path)
    
    # Download the file
    response = requests.get(file_url)
    
    # Save the file to the specified directory
    save_path = os.path.join(save_directory, filename)
    with open(save_path, 'wb') as f:
        f.write(response.content)
    
    # Change the link to the local file
    local_link = os.path.join('resources', filename)
    link['href'] = local_link

# Save the modified HTML file
modified_html_file = os.path.join(save_directory, 'modified_distance_map.html')
with open(modified_html_file, 'w') as f:
    f.write(str(soup))

In [24]:
gdf

,地点,经度,纬度,Table Name,geometry
0,后店村,116.251900,40.547000,丰宁县-01,POINT (116.25190 40.54700)
1,富贵山村,116.221700,41.218900,丰宁县-02,POINT (116.22170 41.21890)
2,九宫号村,116.175900,41.032500,丰宁县-03,POINT (116.17590 41.03250)
3,高栅子村,116.211000,40.595900,丰宁县-04,POINT (116.21100 40.59590)
4,杨木栅子村,116.223900,40.581200,丰宁县-05,POINT (116.22390 40.58120)
...,...,...,...,...,...
259,木兰林场北沟分场黄家东沟,117.481875,41.836140,木兰围场林场-41,POINT (117.48188 41.83614)
260,木兰林场八英庄分场光顶山林路旁,117.948071,42.020240,木兰围场林场-42,POINT (117.94807 42.02024)
261,木兰林场八英庄分场贾家梁,117.864605,42.066896,木兰围场林场-43,POINT (117.86460 42.06690)
262,木兰林场八英庄分场下窝铺,117.869296,42.054027,木兰围场林场-44,POINT (117.86930 42.05403)


In [25]:
gdf.to_crs(epsg=4548, inplace=True)

In [26]:
gdf

,地点,经度,纬度,Table Name,geometry
0,后店村,116.251900,40.547000,丰宁县-01,POINT (436629.231 4490536.802)
1,富贵山村,116.221700,41.218900,丰宁县-02,POINT (434734.441 4565175.559)
2,九宫号村,116.175900,41.032500,丰宁县-03,POINT (430697.928 4544509.755)
3,高栅子村,116.211000,40.595900,丰宁县-04,POINT (433213.239 4495997.227)
4,杨木栅子村,116.223900,40.581200,丰宁县-05,POINT (434290.814 4494355.122)
...,...,...,...,...,...
259,木兰林场北沟分场黄家东沟,117.481875,41.836140,木兰围场林场-41,POINT (540026.024 4633548.940)
260,木兰林场八英庄分场光顶山林路旁,117.948071,42.020240,木兰围场林场-42,POINT (578523.887 4654319.910)
261,木兰林场八英庄分场贾家梁,117.864605,42.066896,木兰围场林场-43,POINT (571558.390 4659429.038)
262,木兰林场八英庄分场下窝铺,117.869296,42.054027,木兰围场林场-44,POINT (571961.145 4658003.573)


In [27]:
gdf['地点']

0                  后店村
1                 富贵山村
2                 九宫号村
3                 高栅子村
4                杨木栅子村
            ...       
259       木兰林场北沟分场黄家东沟
260    木兰林场八英庄分场光顶山林路旁
261       木兰林场八英庄分场贾家梁
262       木兰林场八英庄分场下窝铺
263       木兰林场八英庄分场牙义沟
Name: 地点, Length: 264, dtype: object

In [29]:
gdf[gdf['地点'] == '木兰林场龙头山分场四区高架桥下沟']['geometry']

219    POINT (563661.366 4648591.034)
Name: geometry, dtype: geometry

In [16]:
xy

,地点,经度,纬度,Table Name,geometry
